# Image Compression using Randomized SVD

In this notebook, we demonstrate the effectiveness of **Randomized SVD (rSVD)** by compressing a high-resolution image. 
We compare the reconstruction quality and performance against the standard deterministic SVD provided by NumPy.

**Goal:** Show that rSVD achieves similar visual quality while working on a much smaller subspace.

In [ ]:
import sys
import os
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy import datasets  # or 'from skimage import data' if you have scikit-image

# Add project root to path to import src
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.core import rsvd

# Load a sample image (Gray scale)
# If scipy is not installed, use any grayscale image loading method
try:
    img = datasets.face(gray=True)
except AttributeError:
    # Fallback for older scipy versions or different environments
    img = datasets.ascent()

print(f"Original Image Shape: {img.shape}")

In [ ]:
def compress_and_reconstruct(matrix, target_rank):
    """
    Helper function to perform rSVD and reconstruct the image.
    """
    start_time = time.time()
    
    # 1. Compute rSVD
    U, S, Vt = rsvd(matrix, t=target_rank)
    
    # 2. Reconstruct (Approximation)
    # X_approx = U * S * Vt
    reconstructed = U @ S @ Vt
    
    elapsed = time.time() - start_time
    
    # Calculate compression ratio (approximate)
    original_size = matrix.size
    compressed_size = U.size + S.size + Vt.size
    ratio = original_size / compressed_size
    
    return reconstructed, elapsed, ratio

# Settings
ranks = [10, 50, 100]  # Different compression levels

In [ ]:
plt.figure(figsize=(15, 5))

# Plot Original
plt.subplot(1, len(ranks) + 1, 1)
plt.imshow(img, cmap='gray')
plt.title("Original")
plt.axis('off')

for i, r in enumerate(ranks):
    rec_img, t_sec, ratio = compress_and_reconstruct(img, target_rank=r)
    
    plt.subplot(1, len(ranks) + 1, i + 2)
    plt.imshow(rec_img, cmap='gray')
    plt.title(f"Rank {r}\nTime: {t_sec:.3f}s\nCompr: {ratio:.1f}x")
    plt.axis('off')

plt.tight_layout()
plt.show()